In [12]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [13]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [14]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(16, 128, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.feature_size = self._get_conv_output((3, 32, 32))

        self.fc1 = nn.Linear(self.feature_size, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def _get_conv_output(self, shape):
        input = torch.rand(1, *shape)
        output = self.forward_features(input)
        return int(np.prod(output.size()))

    def forward_features(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = x.view(-1, self.feature_size)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x


In [16]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [17]:
train_losses = []
valid_losses = []
train_accuracies = []
valid_accuracies = []

num_epochs = 10
for epoch in range(num_epochs):
    net.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
    
    train_losses.append(running_loss / (i + 1))
    train_accuracies.append(100 * correct_train / total_train)
    
    net.eval()
    running_loss = 0.0
    correct_valid = 0
    total_valid = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_valid += labels.size(0)
            correct_valid += (predicted == labels).sum().item()
    
    valid_losses.append(running_loss / len(testloader))
    valid_accuracies.append(100 * correct_valid / total_valid)
    
    print(f'Epoch {epoch+1}, Train Loss: {train_losses[-1]:.3f}, Train Acc: {train_accuracies[-1]:.2f}%, Val Loss: {valid_losses[-1]:.3f}, Val Acc: {valid_accuracies[-1]:.2f}%')

net.eval()
correct_test = 0
total_test = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print(f'Test Accuracy: {test_accuracy:.2f}%')

Epoch 1, Train Loss: 1.676, Train Acc: 38.22%, Val Loss: 1.296, Val Acc: 53.53%
Epoch 2, Train Loss: 1.180, Train Acc: 58.16%, Val Loss: 1.120, Val Acc: 61.09%
Epoch 3, Train Loss: 0.983, Train Acc: 65.51%, Val Loss: 1.034, Val Acc: 65.06%
Epoch 4, Train Loss: 0.862, Train Acc: 69.82%, Val Loss: 0.925, Val Acc: 68.55%
Epoch 5, Train Loss: 0.769, Train Acc: 73.30%, Val Loss: 0.938, Val Acc: 68.22%
Epoch 6, Train Loss: 0.687, Train Acc: 76.00%, Val Loss: 0.963, Val Acc: 68.73%
Epoch 7, Train Loss: 0.622, Train Acc: 78.22%, Val Loss: 0.986, Val Acc: 68.20%
Epoch 8, Train Loss: 0.559, Train Acc: 80.36%, Val Loss: 0.939, Val Acc: 69.54%
Epoch 9, Train Loss: 0.520, Train Acc: 81.61%, Val Loss: 1.015, Val Acc: 68.80%
Epoch 10, Train Loss: 0.480, Train Acc: 83.18%, Val Loss: 1.062, Val Acc: 68.68%
Test Accuracy: 68.68%
